# **Running the BAO fit for a given dataset and settings**

This notebook shows how to run a BAO fit

In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
import numpy as np
from scipy.optimize import curve_fit
from utils_baofit import WThetaModel, BAOFitInitializer, BAOFit

# 1. Arguments
class Args:
    def __init__(self):
        self.include_wiggles = "y"
        self.dataset = "DESY6"
        self.weight_type = 1
        self.nz_flag = "fid"
        self.cov_type = "cosmolike_data"
        self.cosmology_template = "planck"
        self.cosmology_covariance = "planck"
        self.delta_theta = 0.2
        self.theta_min = 0.5
        self.theta_max = 5
        self.n_broadband = 3
        self.bins_removed = 'None'
        self.diag_only = "n"
        self.remove_crosscov = "n"
args = Args()
args.include_wiggles = '' if args.include_wiggles == 'y' else '_nowiggles'

# 2. Galaxy bias dictionary
galaxy_bias = {0: 1.7984801248816065, 1: 1.8023169145559157, 2: 1.8102780823227076, 3: 1.9542868433512404, 4: 2.1103760953498414, 5: 2.4099120180419513}
# galaxy_bias = {bin_z: 1.5 for bin_z in range(10)}

# 3. BAO fit initializer
baofit_initializer = BAOFitInitializer(
    include_wiggles=args.include_wiggles,
    dataset=args.dataset,
    weight_type=args.weight_type,
    nz_flag=args.nz_flag,
    cov_type=args.cov_type,
    cosmology_template=args.cosmology_template,
    cosmology_covariance=args.cosmology_covariance,
    delta_theta=args.delta_theta,
    theta_min=args.theta_min,
    theta_max=args.theta_max,
    n_broadband=args.n_broadband,
    bins_removed=args.bins_removed,
    verbose=True
)
args.bins_removed = baofit_initializer.bins_removed # it has been post-processed
nbins = baofit_initializer.nbins

# 4. Load data w(theta) and covariance
from utils_data import WThetaDataCovariance
wtheta_data_covariance = WThetaDataCovariance(
    dataset=args.dataset,
    weight_type=args.weight_type,
    cov_type=args.cov_type,
    cosmology_covariance=args.cosmology_covariance,
    delta_theta=args.delta_theta,
    theta_min=args.theta_min,
    theta_max=args.theta_max,
    bins_removed=args.bins_removed,
    diag_only=args.diag_only,
    remove_crosscov=args.remove_crosscov,
    nbins=nbins
)
theta_data, wtheta_data, cov = wtheta_data_covariance.process()

# 5. Get the template wtheta
wtheta_model = WThetaModel(
    dataset=args.dataset,
    include_wiggles=args.include_wiggles,
    nz_flag=args.nz_flag,
    cosmology_template=args.cosmology_template,
    n_broadband=args.n_broadband,
    galaxy_bias=galaxy_bias, 
)

# THIS DOES NOT WORK ANYMORE
# # --. First fit attempt. We use curve_fit.
# wtheta_data_concatenated = np.concatenate([wtheta_data[bin_z] for bin_z in range(nbins)])
# n_params = len(wtheta_model.names_params)
# n_params_true = len(wtheta_model.names_params) - (1 + args.n_broadband) * len(args.bins_removed)
# wtheta_template = wtheta_model.get_wtheta_template()

# popt, pcov = curve_fit(wtheta_template, theta_data, wtheta_data_concatenated, p0=wtheta_model.p0, bounds=wtheta_model.bounds, sigma=cov, absolute_sigma=False)
# err_params = np.sqrt(np.diag(pcov))

# # Let's compute the chi2 and dof
# wtheta_fit_concatenated = wtheta_template(theta_data, *popt)
# diff = wtheta_data_concatenated - wtheta_fit_concatenated
# chi_square = diff @ np.linalg.inv(cov) @ diff
# dof = len(wtheta_data_concatenated) - n_params_true
# for bin_z in args.bins_removed:
#     dof -= len(indices_theta_allbins[bin_z])

# print(f'Best-fit alpha = {popt[0]:.4f} ± {np.sqrt(pcov[0, 0]):.4f}')
# print(f'chi2/dof = {chi_square:.4f}/{dof}')

# 6. With our pipeline
baofit = BAOFit(
    baofit_initializer=baofit_initializer,
    wtheta_model=wtheta_model,
    theta_data=theta_data,
    wtheta_data=wtheta_data,
    cov=cov,
    n_cpu=128
)
alpha_best, err_alpha, chi2_best, dof = baofit.fit()


Saving output to: fit_results/DESY6_1/nzfid_covcosmolike_data_plancktemp_planckcov_deltatheta0.2_thetamin0.5_thetamax5_3broadband_binsremoved[]
All theta values are the same. Using the first one as an array.
Using DESY6 fid n(z), which has 6 redshift bins
Best-fit alpha = 0.9676 ± 0.0249
chi2/dof = 91.8341/107
